Sailing The Seaborn! Below are the majority 4 topics here

1. Examine call volume and identify surge times, seasons, or events.

Let's try and do this by looking at peak hours and days. Then with that info we can look to see if there was a specific event that occured then? Seasonality or looking at sunrise/sunset times would be helpful as well.

2. Analyze trends in call content by looking for which issues are most frequent and what is the average count of issues per call.

This will likely be regex heavy? Would be searching for keywords within the call dialogue.

3. Assess regional and demographic trends in call content and call volume.

This is just like doing #2 but including demographics

4. Determine the count and success rate of imminent risk calls.

This seems the easiest, would be used best when paired with the other means of looking into the data

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

I am importing these items as it is fortold in the python commandments

In [ ]:
crisis1 = pd.read_excel('../data/NSSCrisisv1.xlsx', header=2)
crisis2 = pd.read_excel('../data/NSSCrisisv2.xlsx', header=2)

In [ ]:
crisis1.head()

I'm converting the call times here to datetime format! Info & formatting pulled from here
https://www.geeksforgeeks.org/python-working-with-date-and-time-using-pandas/

In [ ]:
crisis1['CallDateAndTimeStart'] = pd.to_datetime(crisis1['CallDateAndTimeStart'])
crisis1['CallDateAndTimeEnd'] = pd.to_datetime(crisis1['CallDateAndTimeEnd'])
crisis1['CallDateStartWeekdate'] = crisis1['CallDateAndTimeStart'].dt.day_name()
#CallDateStartWeekdate give a Monday, Tuesday etc result
crisis1['CallDateStartTime'] = crisis1['CallDateAndTimeStart'].dt.hour
#CallDateStartTime gives the hour of the day the call was made, military time
crisis1['CallDateStartMonth'] = crisis1['CallDateAndTimeStart'].dt.month
#CallDateStartMonth gives the month which the call was made
crisis1['CallDateAndTimeStart']

Attempting to do a regex here so that I can make a kde seaborn plot for call length and call time of day, but I'm having an issue here of pulling the call time. I tried \d{2}:\d{2}:00 but that was coming up with nothing, somehow. I don't think colon is a strange character though? https://seaborn.pydata.org/generated/seaborn.kdeplot.html?highlight=kde%20plot#seaborn.kdeplot https://towardsdatascience.com/an-ultimate-cheatsheet-of-data-visualization-in-seaborn-be8ed13a3697 https://seaborn.pydata.org/tutorial/relational.html https://www.dataquest.io/wp-content/uploads/2019/03/python-regular-expressions-cheat-sheet.pdf

In [ ]:
#crisis1['CallDateAndTimeStart'] = crisis1['CallDateAndTimeStart'].astype('str')
#crisis1['CallDateStartTime'] = crisis1['CallDateAndTimeStart'].str.findall(r'\d{2}:\d{2}:00')
#crisis1['CallDateStartDate'] = crisis1['CallDateAndTimeStart'].str.findall(r'\d{4}-\d{2}-\d{2}')
#crisis1['CallDateStartDate']
#crisis1['CallDateStartTime']

In [ ]:
crisis1.info()

In [ ]:
crisis1.describe(include='all')

In [ ]:
crisis1.dropna(how='all', axis=1, inplace=True)

In [ ]:
for label, content in crisis1.items():
    print(f'label: {label}')
    print(f'content: {content}', sep='\n')

In [ ]:
sns.barplot(data=crisis1, x="CallLength", y="CallDateStartWeekdate", 
            #hue="CRISIS Demographics - Gender Identity"
           )

Mondays am i right lol

In [ ]:
g = sns.FacetGrid(
    crisis1, 
    col="CallDateStartWeekdate",
    row="CRISIS Demographics - Gender Identity"
    )
g.map_dataframe(sns.histplot, x="CallLength")
#g.fig.set_size_inches(18, 10)
#g.set_axis_labels("Total Bill", "Frequency")

In [ ]:
sns.boxplot(data=crisis1, x="CallLength", y="CallDateStartWeekdate", showfliers = False)
#no outliers call time chart

In [ ]:
sns.boxplot(data=crisis1, x="CallDateStartTime", y="CallLength", showfliers = False)

Early morning (sunrise) has much lower call volume, while ~sunset~ tends to have more calls

In [ ]:
sns.histplot(data=crisis1, x="CallDateStartTime")

In [ ]:
sns.scatterplot(x="CallDateAndTimeStart", y="CallLength", data=crisis1)

nvm guess this heatmap doesn't quite work for what i'm attempting

In [ ]:
sns.heatmap(crisis1['CallDateStartMonth'], crisis1['CallDateStartWeekdate'], crisis1['CallLength'], nnot=True, fmt="d")

In [ ]:
crisis1['CallDateStartWeekdate']